In [1]:
using Plots
using JLD
using StatsPlots
using LaTeXStrings


include("../src/FCSeqTools.jl");
include("../src/functions.jl");

folder_name = "pseudo_count=0.5__init_pseudo_count=0.01"

rm(joinpath("figures/comparison_pseudocount",folder_name), force=true, recursive=true)
mkdir(joinpath("figures/comparison_pseudocount",folder_name))
path = joinpath("figures/comparison_pseudocount",folder_name);

# Compare Couplings

In [7]:
method = "cumulative"
fraction = 0.3
stop = 0.9
pseudo_count = 0.5

path1 = "../training/" * method * string(fraction) * "_stop=" * string(stop) * "_reg="*string(pseudo_count) #* "_h_ps-count=" * string(init_pseudo_count)
path2 = "../training/" * method * string(fraction) * "_stop=" * string(stop)  #* "_h_ps-count=" * string(init_pseudo_count)

h1 = JLD.load(path1* "/h.jld")["data"]
Jij1 = JLD.load(path1* "/Jij.jld")["data"];
iteration1 = open(path1* "/iterations.txt", "r")
data1 = parse.(Float64, split(readlines(iteration1)[end], " ")[1:end-1])

h2 = JLD.load(path2* "/h.jld")["data"]
Jij2 = JLD.load(path2* "/Jij.jld")["data"];
iteration2 = open(path2* "/iterations.txt", "r");
data2 = parse.(Float64, split(readlines(iteration2)[end], " ")[1:end-1])

edge_n1, element_n1 = data1[6], data1[7]
edge_n2, element_n2 = data2[6], data2[7]
Nij_tot = 96*(96-1)
Nij_ab_tot = Nij_tot * q^2;

In [8]:
n_element_plot = 500_000
Jij1_to_plot = zeros(n_element_plot);
Jij2_to_plot = zeros(n_element_plot);

for i in 1:n_element_plot
    j = rand(1:length(Jij1))
    Jij1_to_plot[i] = Jij1[j]
    Jij2_to_plot[i] = Jij2[j]
end

In [9]:
scatter(Jij1_to_plot, Jij2_to_plot, markershape=:circle, label=:false, xlabel="Jij(a,b) α = 0.5", ylabel="Jij(a,b) α = 0.05", title="All the couplings", alpha=0.2)
savefig("figures/comparison_pseudocount/Jij.png");

In [24]:
n_ij, n_ij_ab, n_01, n_10, n_00, Jij_common1, Jij_common2 = count_common_Jij(Jij1, Jij2);

frac_ij_tot = round(n_ij/Nij_tot * 100, digits=1)
frac_ijab_tot = round(n_ij_ab/Nij_ab_tot * 100, digits=1)

frac_ij_1 = round(n_ij/edge_n1 * 100, digits=1)
frac_ijab_1 = round(n_ij_ab/element_n1 * 100, digits=1)
frac_ij_2 = round(n_ij/edge_n2 * 100, digits=1)
frac_ijab_2 = round(n_ij_ab/element_n2 * 100, digits=1)

println("Common Edges: \n",frac_ij_tot,"% of the total;   ",  frac_ij_1, "% of the first model;   ", frac_ij_2, "% of the second model")
println("\nCommon Elements: \n",frac_ijab_tot,"% of the total;   ",  frac_ijab_1, "% of the first model;   ", frac_ijab_2, "% of the second model")
println("\nNumber of cases where: J_0.5 == 0 &  J_{0.05} != 0: ", n_01)
println("\nNumber of cases where: J_0.5 != 0 &  J_{0.05} == 0: ", n_10)
println("\nNumber of cases where: J_0.5 == 0 &  J_{0.05} == 0: ", n_00)
println("\nNumber of cases where: J_0.5 != 0 &  J_{0.05} != 0: ", n_ij_ab)


Common Edges: 
14.3% of the total;   79.9% of the first model;   77.1% of the second model

Common Elements: 
0.2% of the total;   41.4% of the first model;   24.4% of the second model

Number of cases where: J_0.5 == 0 &  J_{0.05} != 0: 27991

Number of cases where: J_0.5 != 0 &  J_{0.05} == 0: 12788

Number of cases where: J_0.5 == 0 &  J_{0.05} == 0: 4014429

Number of cases where: J_0.5 != 0 &  J_{0.05} != 0: 9048


In [11]:
x, y = Jij_common1[Jij_common1.!= 0.0], Jij_common2[Jij_common2.!= 0.0]
x_label, y_label = L"J_{ij}(a,b), \alpha=0.5", L"J_{ij}(a,b), \alpha=0.05"
title = "Common Couplings: "* L"J_{ij}(a,b)"


plot(  histogram(x, nbins=100, alpha=0.5, orientation=:vertical,label=:false, xticks=[], ylabel="counts", title = title),   
       plot(framestyle = :none),
       scatter(x, y, markershape=:circle, label=:false, xlabel=x_label, ylabel=y_label, alpha=0.2),
       histogram(y, nbins=50, alpha=0.5, orientation=:horizontal, label=:false, yticks=[], xlabel="counts"),
       link = :both, 
       size=(1000, 1000), 
       spacing=0.1, 
       )

savefig("figures/comparison_pseudocount/Common_Jij.png");

# test 